# Architectures

> Scripts to get architectures for the models

In [29]:
#| default_exp architectures

In [30]:
#| export
#| hide

import torch
import torch.nn as nn
from abc import ABC, abstractmethod

## Sampling

In [31]:
#| export
class Sampling(nn.Module):
    """
    PyTorch module to perform the reparameterization trick for VAEs.
    """
    def forward(self, z_mean, z_log_var):
        std = torch.exp(0.5 * z_log_var)
        eps = torch.randn_like(std)
        return z_mean + eps * std

## Conv5 Architecture

In [32]:
#| export
class Conv5Encoder(nn.Module):
    def __init__(self, seq_len, feat_dim, latent_dim, dropout_rate):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        
        # Group 1: Transpose Convolution Layers to reshape and refine the spatial structure
        self.convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim, out_channels=64, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=7, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=13, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Flatten()
        )
        # Group 2: Dense Layers to downscale the feature space to a lower latent dimensions
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)  # Output z_mean and z_log_var
        )

        self.sampling = Sampling()

    def forward(self, x):
        x = self.convo_layers(x)
        x = self.dense_layers(x)
        z_mean = x[:, :self.latent_dim]
        z_log_var = x[:, self.latent_dim:]
        z = self.sampling(z_mean, z_log_var)
        return z, z_mean, z_log_var
    


In [33]:
#| export
class Conv5Decoder(nn.Module):
    def __init__(self, seq_len, feat_dim, latent_dim, dropout_rate):
        super().__init__()
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        
        # Group 1: Dense Layers to upscale the latent dimensions to a higher feature space
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * self.seq_len),
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, self.seq_len))
        )

        # Group 2: Transpose Convolution Layers to reshape and refine the spatial structure
        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=13, stride=1, padding=6),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding=4),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=7, stride=1, padding=3),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, z):
        z = self.dense_layers(z)
        z = self.conv_transpose_layers(z)
        return z

In [34]:
#| export
def get_conv5_vae_components(seq_len, feat_dim, latent_dim, dropout_rate=0.2):
    """
    Returns an instance of Conv5Encoder and Conv5Decoder based on the given parameters.
    
    Args:
        seq_len (int): Length of input sequence.
        feat_dim (int): Dimensionality of input features.
        latent_dim (int): Dimensionality of the latent space.
        dropout_rate (float): Dropout rate to use in the model.
    
    Returns:
        encoder (Conv5Encoder): The encoder part of the VAE.
        decoder (Conv5Decoder): The decoder part of the VAE.
    """
    encoder = Conv5Encoder(seq_len, feat_dim, latent_dim, dropout_rate)
    decoder = Conv5Decoder(seq_len, feat_dim, latent_dim, dropout_rate)
    
    return encoder, decoder

In [35]:
#| hide
import nbdev; nbdev.nbdev_export()